In [7]:
! pip install gym
! git clone https://github.com/gyyang/neurogym.git
%cd neurogym/
! pip install -e .

fatal: destination path 'neurogym' already exists and is not an empty directory.
/Users/mayasrikm/Desktop/Code/rl/ORCS4529-Neuro-Project/neurogym
Obtaining file:///Users/mayasrikm/Desktop/Code/rl/ORCS4529-Neuro-Project/neurogym
  Preparing metadata (setup.py) ... done
  Attempting uninstall: neurogym
    Found existing installation: neurogym 0.0.2
    Uninstalling neurogym-0.0.2:
      Successfully uninstalled neurogym-0.0.2
  Running setup.py develop for neurogym


In [11]:
%pip install numpy --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 9.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.2 which is incompatible.
scipy 1.9.1 requires numpy<1.25.0,>=1.18.5, but you have numpy 1.26.2 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Environment
from tasks import PerceptualDecisionMaking
env = PerceptualDecisionMaking()
_ = env.reset()
ob_size = env.observation_space.shape[0]
act_size = env.action_space.n

In [65]:
import numpy as np
from tqdm import tqdm
episodes = 200
epsilon = 0.1
gamma=0.9
_ = env.reset()
alpha = 0.1
Q = np.zeros((ob_size, act_size))
phi = np.zeros((1, ob_size)) 
w = np.random.randn(ob_size) 
for i in tqdm(range(episodes)):
  trial = env.new_trial()
  phi= env.ob.flatten()
  if np.max(Q) == 0 or np.random.rand() < epsilon:
      action = env.action_space.sample()
  else:
      action = np.argmax(Q) % act_size
  V = np.multiply(phi,w) 
  Q[:, action] = Q[:, action]+alpha*V
     
  phi, r, done, _ = env.step(action)
  phi = phi.flatten()
  rpe = r+gamma*(np.multiply(phi, w)-V) 
  w = w + alpha*phi*rpe  

  0%|          | 0/200 [00:00<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (66,) (3,) 